In [ ]:
import torch
import os
import numpy as np
import pandas as pd

In [2]:
MODEL_PATH = "/data/renhaoye/decals_2022/trained_model/x_ception-LR_0.0001-LOSS_focal_loss-CLASS_7-BATCHSIZE_32-OPTIM_torch.optim.AdamW-OTHER_graduation_weightdecay0.01/model_5.pt"

In [6]:
sdss_dir = "/data/renhaoye/sdss_dr7_decals_overlap/scaled/"
decals_csv = "/data/renhaoye/decals_2022/fits.csv"
sdss_files = os.listdir(sdss_dir)
decals_df = pd.read_csv(decals_csv)
print(len(sdss_files), decals_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313382 entries, 0 to 313381
Data columns (total 92 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   Unnamed: 0                                         313382 non-null  int64  
 1   ra                                                 313382 non-null  float64
 2   dec                                                313382 non-null  float64
 3   iauname                                            313382 non-null  object 
 4   petro_theta                                        313382 non-null  float64
 5   petro_th50                                         313382 non-null  float64
 6   petro_th90                                         313382 non-null  float64
 7   elpetro_absmag_r                                   313378 non-null  float64
 8   sersic_nmgy_r                                      313382 non-null  float6

In [7]:
from sklearn.model_selection import train_test_split
threshold = 0.5
threshold2 = 0.6
merger = decals_df.query('merging_minor_disturbance_fraction > %f '
                               '| merging_major_disturbance_fraction > %f '
                               '| merging_merger_fraction > %f '
                               % (threshold2, threshold2, threshold2))
smoothRounded = decals_df.query('smooth_or_featured_smooth_fraction >  %f '
                              '& how_rounded_round_fraction > %f' % (0.7, 0.8))
smoothInBetween = decals_df.query('smooth_or_featured_smooth_fraction >  %f '
                                '& how_rounded_in_between_fraction > %f' % (0.7, 0.85))
smoothCigarShaped = decals_df.query('smooth_or_featured_smooth_fraction > %f '
                                  '& how_rounded_cigar_shaped_fraction > %f' % (threshold, threshold2))
edgeOn = decals_df.query('smooth_or_featured_featured_or_disk_fraction > %f '
                       '& disk_edge_on_yes_fraction > %f'
                       % (threshold, 0.7))
diskNoBar = decals_df.query('smooth_or_featured_featured_or_disk_fraction > %f '
                          '& disk_edge_on_no_fraction > %f '
                          '& bar_no_fraction > %f '
                          % (threshold, threshold, 0.7))
diskStrongBar = decals_df.query('smooth_or_featured_featured_or_disk_fraction > %f '
                              '& disk_edge_on_no_fraction > %f '
                              '&bar_strong_fraction > %f '
                              % (threshold, threshold, threshold2))
merger.loc[:,"label"] = 0
smoothRounded.loc[:,"label"] = 1
smoothInBetween.loc[:,"label"] = 2
smoothCigarShaped.loc[:,"label"] = 3
edgeOn.loc[:,"label"] = 4
diskNoBar.loc[:,"label"] = 5
diskStrongBar.loc[:,"label"] = 6
merger_train, merger_test = train_test_split(merger, test_size=0.1, random_state=1926)
merger_train, merger_valid = train_test_split(merger_train, test_size=0.1, random_state=1926)

smoothRounded_train, smoothRounded_test = train_test_split(smoothRounded, test_size=0.1, random_state=1926)
smoothRounded_train, smoothRounded_valid = train_test_split(smoothRounded_train, test_size=0.1, random_state=1926)

smoothInBetween_train, smoothInBetween_test = train_test_split(smoothInBetween, test_size=0.1, random_state=1926)
smoothInBetween_train, smoothInBetween_valid = train_test_split(smoothInBetween_train, test_size=0.1,
                                                                random_state=1926)

smoothCigarShaped_train, smoothCigarShaped_test = train_test_split(smoothCigarShaped, test_size=0.1,
                                                                   random_state=1926)
smoothCigarShaped_train, smoothCigarShaped_valid = train_test_split(smoothCigarShaped_train, test_size=0.1,
                                                                    random_state=1926)

edgeOn_train, edgeOn_test = train_test_split(edgeOn, test_size=0.1, random_state=1926)
edgeOn_train, edgeOn_valid = train_test_split(edgeOn_train, test_size=0.1, random_state=1926)

diskNoBar_train, diskNoBar_test = train_test_split(diskNoBar, test_size=0.1, random_state=1926)
diskNoBar_train, diskNoBar_valid = train_test_split(diskNoBar_train, test_size=0.1, random_state=1926)

diskStrongBar_train, diskStrongBar_test = train_test_split(diskStrongBar, test_size=0.1, random_state=1926)
diskStrongBar_train, diskStrongBar_valid = train_test_split(diskStrongBar_train, test_size=0.1, random_state=1926)

/home/renhaoye/miniconda3/envs/decals/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [8]:
merger_train = merger_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
merger_test = merger_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
merger_valid = merger_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
merger_train.loc[:, "func"] = "train"
merger_test.loc[:, "func"] = "test"
merger_valid.loc[:, "func"] = "valid"

smoothRounded_train = smoothRounded_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothRounded_test = smoothRounded_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothRounded_valid = smoothRounded_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothRounded_train.loc[:, "func"] = "train"
smoothRounded_test.loc[:, "func"] = "test"
smoothRounded_valid.loc[:, "func"] = "valid"

smoothInBetween_train = smoothInBetween_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothInBetween_test = smoothInBetween_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothInBetween_valid = smoothInBetween_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothInBetween_train.loc[:, "func"] = "train"
smoothInBetween_test.loc[:, "func"] = "test"
smoothInBetween_valid.loc[:, "func"] = "valid"

smoothCigarShaped_train = smoothCigarShaped_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothCigarShaped_test = smoothCigarShaped_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothCigarShaped_valid = smoothCigarShaped_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
smoothCigarShaped_train.loc[:, "func"] = "train"
smoothCigarShaped_test.loc[:, "func"] = "test"
smoothCigarShaped_valid.loc[:, "func"] = "valid"

edgeOn_train = edgeOn_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
edgeOn_test = edgeOn_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
edgeOn_valid = edgeOn_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
edgeOn_train.loc[:, "func"] = "train"
edgeOn_test.loc[:, "func"] = "test"
edgeOn_valid.loc[:, "func"] = "valid"

diskNoBar_train = diskNoBar_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskNoBar_test = diskNoBar_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskNoBar_valid = diskNoBar_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskNoBar_train.loc[:, "func"] = "train"
diskNoBar_test.loc[:, "func"] = "test"
diskNoBar_valid.loc[:, "func"] = "valid"

diskStrongBar_train = diskStrongBar_train.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskStrongBar_test = diskStrongBar_test.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskStrongBar_valid = diskStrongBar_valid.loc[:,["ra", "dec", "redshift", "iauname", "petro_theta", "petro_th50", "petro_th90", "elpetro_absmag_r", "mag_r", "label"]]
diskStrongBar_train.loc[:, "func"] = "train"
diskStrongBar_test.loc[:, "func"] = "test"
diskStrongBar_valid.loc[:, "func"] = "valid"

In [9]:
df = pd.concat([merger_train, merger_test, merger_valid, smoothRounded_train, smoothRounded_test, smoothRounded_valid, smoothInBetween_train, smoothInBetween_test, smoothInBetween_valid, smoothCigarShaped_train, smoothCigarShaped_test, smoothCigarShaped_valid, edgeOn_train, edgeOn_test, edgeOn_valid, diskNoBar_train, diskNoBar_test, diskNoBar_valid, diskStrongBar_train, diskStrongBar_test, diskStrongBar_valid])

In [24]:
df.loc[:, 'fname'] = "nan"
for i in range(len(df)):
    line = str(df.iloc[i].ra) + "_" + str(df.iloc[i].dec) + ".fits"
    df.loc[i, 'fname'] = line

In [25]:
df

,ra,dec,redshift,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,mag_r,label,func,fname
280684,228.253553,17.001007,0.128372,J151300.84+170003.6,6.97497,3.530749,8.043182,-21.39746,16.741436,0,train,nan
308571,18.403378,1.242624,0.120664,J011336.81+011433.4,5.357219,2.873933,7.314268,-21.831284,16.062775,0,train,nan
268979,356.932802,-2.610917,0.030901,J234744.50-023646.9,15.164274,5.864809,18.633486,-19.064878,15.318168,0,train,nan
296601,215.997519,-0.890637,0.133594,J142359.39-005326.2,10.527441,4.22471,14.072979,-22.040115,16.286236,0,train,nan
174192,155.049169,14.096121,0.146089,J102011.79+140546.1,9.513814,5.17791,12.710546,-22.37852,15.564474,0,train,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
123947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.08812_2.786511.fits
123948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.016286_-0.988534.fits
123949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.653104_-4.682799.fits
123953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.5398_18.409219.fits


In [28]:
df.to_csv("/data/renhaoye/decals_2022/dataset_decals.csv")

In [29]:
x = pd.read_csv("/data/renhaoye/decals_2022/dataset_decals.csv", index_col=0)

In [49]:
x.loc[:, 'fname'] = x.loc[:, 'ra'].astype(str) + "_" + x.loc[:, 'dec'] + ".fits"

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [48]:
x.iloc[1].fname

'280684    228.253553\n308571     18.403378\n268979    356.932802\n296601    215.997519\n174192    155.049169\n             ...    \n219841     24.653104\n110592    243.579343\n152632     17.974347\n25118      32.648135\n299065    156.539800\nName: ra, Length: 123954, dtype: float64_280684    17.001007\n308571     1.242624\n268979    -2.610917\n296601    -0.890637\n174192    14.096121\n            ...    \n219841    -4.682799\n110592    30.214586\n152632    -4.618883\n25118     -0.937623\n299065    18.409219\nName: dec, Length: 123954, dtype: float64.fits'

In [40]:
x.fname = x.fname.map(lambda x: str(x.ra) + "_" + str(x.dec) + ".fits")

AttributeError: 'str' object has no attribute 'ra'

In [ ]:
x

In [38]:
for i in x:
    print(i)
    # line = str(i.ra) + "_" + str(i.dec) + ".fits"
    # i.fname = line

ra
dec
redshift
iauname
petro_theta
petro_th50
petro_th90
elpetro_absmag_r
mag_r
label
func
fname


In [28]:
x

,ra,dec,redshift,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,mag_r,label,func,fname
280684,228.253553,17.001007,0.128372,J151300.84+170003.6,6.974970,3.530749,8.043182,-21.397460,16.741436,0.0,train,nan
308571,18.403378,1.242624,0.120664,J011336.81+011433.4,5.357219,2.873933,7.314268,-21.831284,16.062775,0.0,train,nan
268979,356.932802,-2.610917,0.030901,J234744.50-023646.9,15.164274,5.864809,18.633486,-19.064878,15.318168,0.0,train,nan
296601,215.997519,-0.890637,0.133594,J142359.39-005326.2,10.527441,4.224710,14.072979,-22.040115,16.286236,0.0,train,nan
174192,155.049169,14.096121,0.146089,J102011.79+140546.1,9.513814,5.177910,12.710546,-22.378520,15.564474,0.0,train,nan_nan.fits
...,...,...,...,...,...,...,...,...,...,...,...,...
199175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan_nan.fits
199177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan_nan.fits
199178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan_nan.fits
199179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan_nan.fits


In [202]:
df

,ra,dec,redshift,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,mag_r,label,func
280684,228.253553,17.001007,0.128372,J151300.84+170003.6,6.974970,3.530749,8.043182,-21.397460,16.741436,0,train
308571,18.403378,1.242624,0.120664,J011336.81+011433.4,5.357219,2.873933,7.314268,-21.831284,16.062775,0,train
268979,356.932802,-2.610917,0.030901,J234744.50-023646.9,15.164274,5.864809,18.633486,-19.064878,15.318168,0,train
296601,215.997519,-0.890637,0.133594,J142359.39-005326.2,10.527441,4.224710,14.072979,-22.040115,16.286236,0,train
174192,155.049169,14.096121,0.146089,J102011.79+140546.1,9.513814,5.177910,12.710546,-22.378520,15.564474,0,train
...,...,...,...,...,...,...,...,...,...,...,...
219841,24.653104,-4.682799,0.048784,J013836.69-044058.0,17.698072,7.415717,20.187800,-21.068200,14.825631,6,valid
110592,243.579343,30.214586,0.094828,J161419.04+301252.5,7.391881,3.314074,8.751869,-21.017754,16.270058,6,valid
152632,17.974347,-4.618883,0.053374,J011153.89-043708.0,11.637828,5.300970,11.823557,-20.770077,15.467412,6,valid
25118,32.648135,-0.937623,0.068009,J021035.54-005615.3,12.270542,5.666851,12.626719,-21.048773,15.494813,6,valid


In [29]:
sdss_list = []
for sdss_file in sdss_files:
    sdss_list.append([sdss_file.split("_")[0], sdss_file.split(".fits")[0].split("_")[1]])
sdss_df = pd.DataFrame(sdss_list, columns=["ra", "dec"], dtype=np.float64)
print(sdss_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220431 entries, 0 to 220430
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ra      220431 non-null  float64
 1   dec     220431 non-null  float64
dtypes: float64(2)
memory usage: 3.4 MB
None


In [30]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [162]:
def match(sdss2match, decals2match):
    sdss = SkyCoord(ra=sdss2match.ra*u.degree, dec=sdss2match.dec*u.degree)
    decals = SkyCoord(ra=decals2match.ra*u.degree, dec=decals2match.dec*u.degree)
    idx, d2d, d3d = sdss.match_to_catalog_sky(decals)
    max_sep = 4 * 0.396127 * u.arcsec
    distance_idx = d2d < max_sep
    # sdss_matches = sdss[distance_idx]   # sdss表中的坐标，顺序和decals_matches相同，和sdss_df顺序不同
    # matches = idx[distance_idx]  # decals表中的坐标，顺序和decals_df不同
    # decals_matches = decals[matches]  # decals表中的坐标，顺序和decals_df不同
    sdss_matches = sdss2match.iloc[distance_idx]
    matches = idx[distance_idx]
    decal_matches = decals2match.iloc[matches]
    test = sdss_matches.loc[:].rename(columns={"ra":"sdss_ra", "dec":"sdss_dec"})
    test.insert(0, 'ID', range(len(test)))
    decal_matches.insert(0, 'ID', range(len(decal_matches)))
    new_df = pd.merge(test, decal_matches, how="inner", on=["ID"])
    return new_df.drop("ID", axis=1)

In [167]:
sdss_matches = match(sdss_df, decals_df)

sdss_merger = match(sdss_df, merger)
sdss_merger_train = match(sdss_df, merger_train)
sdss_merger_valid = match(sdss_df, merger_valid)
sdss_merger_test = match(sdss_df, merger_test)

sdss_smoothRounded = match(sdss_df, smoothRounded)
sdss_smoothRounded_train = match(sdss_df, smoothRounded_train)
sdss_smoothRounded_valid = match(sdss_df, smoothRounded_valid)
sdss_smoothRounded_test = match(sdss_df, smoothRounded_test)

sdss_smoothInBetween = match(sdss_df, smoothInBetween)
sdss_smoothInBetween_train = match(sdss_df, smoothInBetween_train)
sdss_smoothInBetween_valid = match(sdss_df, smoothInBetween_valid)
sdss_smoothInBetween_test = match(sdss_df, smoothInBetween_test)

sdss_smoothCigarShaped = match(sdss_df, smoothCigarShaped)
sdss_smoothCigarShaped_train = match(sdss_df, smoothCigarShaped_train)
sdss_smoothCigarShaped_valid = match(sdss_df, smoothCigarShaped_valid)
sdss_smoothCigarShaped_test = match(sdss_df, smoothCigarShaped_test)

sdss_edgeOn = match(sdss_df, edgeOn)
sdss_edgeOn_train = match(sdss_df, edgeOn_train)
sdss_edgeOn_valid = match(sdss_df, edgeOn_valid)
sdss_edgeOn_test = match(sdss_df, edgeOn_test)

sdss_diskNoBar = match(sdss_df, diskNoBar)
sdss_diskNoBar_train = match(sdss_df, diskNoBar_train)
sdss_diskNoBar_valid = match(sdss_df, diskNoBar_valid)
sdss_diskNoBar_test = match(sdss_df, diskNoBar_test)

sdss_diskStrongBar = match(sdss_df, diskStrongBar)
sdss_diskStrongBar_train = match(sdss_df, diskStrongBar_train)
sdss_diskStrongBar_valid = match(sdss_df, diskStrongBar_valid)
sdss_diskStrongBar_test = match(sdss_df, diskStrongBar_test)

In [193]:
def add0(x):
    length = len(x.split(".")[1])
    return x + (6 - length) * "0"

def write_loc(sdss_catalog):
    for row in range(len(sdss_catalog)):
        ra, dec = str(sdss_catalog.loc[row].sdss_ra), str(sdss_catalog.loc[row].sdss_dec)
        line11 = sdss_dir + ra + "_" + dec + ".fits"
        line00 = sdss_dir + add0(ra) + "_" + add0(dec) + ".fits"
        line10 = sdss_dir + ra + "_" + add0(dec) + ".fits"
        line01 = sdss_dir + add0(ra) + "_" + dec + ".fits"
        if os.path.exists(line11):
            sdss_catalog.loc[row, "sdss_loc"] = line11
            # print("11")
        elif os.path.exists(line00):
            sdss_catalog.loc[row, "sdss_loc"] = line00
        elif os.path.exists(line10):
            sdss_catalog.loc[row, "sdss_loc"] = line10
        elif os.path.exists(line01):
            sdss_catalog.loc[row, "sdss_loc"] = line01
        else:
            print(line11)

In [198]:
dataset = [sdss_merger_train, sdss_merger_test, sdss_merger_valid,
                        sdss_smoothRounded_train, sdss_smoothRounded_test, sdss_smoothRounded_valid,
                        sdss_smoothInBetween_train, sdss_smoothInBetween_test, sdss_smoothInBetween_valid,
                        sdss_smoothCigarShaped_train, sdss_smoothCigarShaped_test, sdss_smoothCigarShaped_valid,
                        sdss_edgeOn_train, sdss_edgeOn_test, sdss_edgeOn_valid,
                        sdss_diskNoBar_train, sdss_diskNoBar_test, sdss_diskNoBar_valid,
                        sdss_diskStrongBar_train, sdss_diskStrongBar_test, sdss_diskStrongBar_valid,
                        ]
for i in dataset:
    write_loc(i)

In [199]:
df_matches = pd.concat(dataset)

In [201]:
df_matches.to_csv("/data/renhaoye/decals_2022/dataset_sdss_match_decals.csv")

In [10]:
print("merger len: ", len(sdss_merger), len(merger))
print("merger test_len: ", len(sdss_merger_test), len(merger_test))
print("smoothRounded len: ", len(sdss_smoothRounded), len(smoothRounded))
print("smoothRounded test_len: ", len(sdss_smoothRounded_test), len(smoothRounded_test))
print("smoothInBetween len: ", len(sdss_smoothInBetween), len(smoothInBetween))
print("smoothInBetween test_len: ", len(sdss_smoothInBetween_test), len(smoothInBetween_test))
print("smoothCigarShaped len: ", len(sdss_smoothCigarShaped), len(smoothCigarShaped))
print("smoothCigarShaped test_len: ", len(sdss_smoothCigarShaped_test), len(smoothCigarShaped_test))
print("edgeOn len: ", len(sdss_edgeOn), len(edgeOn))
print("edgeOn test_len: ", len(sdss_edgeOn_test), len(edgeOn_test))
print("diskNoBar len: ", len(sdss_diskNoBar), len(diskNoBar))
print("diskNoBar test_len: ", len(sdss_diskNoBar_test), len(diskNoBar_test))
print("diskStrongBar len: ", len(sdss_diskStrongBar), len(diskStrongBar))
print("diskStrongBar test_len: ", len(sdss_diskStrongBar_test), len(diskStrongBar_test))

merger len:  2810 4178
merger test_len:  275 418
smoothRounded len:  20964 30541
smoothRounded test_len:  2095 3055
smoothInBetween len:  22591 32790
smoothInBetween test_len:  2301 3279
smoothCigarShaped len:  14638 21834
smoothCigarShaped test_len:  1459 2184
edgeOn len:  9284 14248
edgeOn test_len:  923 1425
diskNoBar len:  12338 18288
diskNoBar test_len:  1236 1829
diskStrongBar len:  1393 2075
diskStrongBar test_len:  142 208


In [45]:
# def add0(x):
#     length = len(x.split(".")[1])
#     return x + (6 - length) * "0"
#
# def write2txt_label(sdss_catalog, label, txt_path):
#     with open(txt_path, "a") as w:
#         for row in sdss_catalog:
#             ra, dec = str(row.ra.degree), str(row.dec.degree)
#             line11 = sdss_dir + ra + "_" + dec + ".fits"
#             line00 = sdss_dir + add0(ra) + "_" + add0(dec) + ".fits"
#             line10 = sdss_dir + ra + "_" + add0(dec) + ".fits"
#             line01 = sdss_dir + add0(ra) + "_" + dec + ".fits"
#             if os.path.exists(line11):
#                 w.write(line11 + " " + str(label) + "\n")
#             elif os.path.exists(line00):
#                 w.write(line00 + " " + str(label) + "\n")
#             elif os.path.exists(line10):
#                 w.write(line10 + " " + str(label) + "\n")
#             elif os.path.exists(line01):
#                 w.write(line01 + " " + str(label) + "\n")
#             else:
#                 print(line11)

In [46]:
# from tqdm import tqdm
# save_path = "/data/renhaoye/decals_2022/sdss_match_decals_label_all.txt"
# w = open(save_path, "w")
# w.close()
# todo = [sdss_merger, sdss_smoothRounded, sdss_smoothInBetween, sdss_smoothCigarShaped, sdss_edgeOn, sdss_diskNoBar, sdss_diskStrongBar]
# for idx in tqdm(range(len(todo))):
#     write2txt_label(todo[idx], idx, save_path)

100%|██████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:51<00:00,  7.39s/it]


In [47]:
# save_path = "/data/renhaoye/decals_2022/sdss_match_decals_label_test.txt"
# w = open(save_path, "w")
# w.close()
# todo = [sdss_merger_test, sdss_smoothRounded_test, sdss_smoothInBetween_test, sdss_smoothCigarShaped_test, sdss_edgeOn_test, sdss_diskNoBar_test, sdss_diskStrongBar_test]
# for idx in tqdm(range(len(todo))):
#     write2txt_label(todo[idx], idx, save_path)

100%|██████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:05<00:00,  1.35it/s]
